![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None)

# Replace '?' with NaN
cc_apps.replace('?', np.nan, inplace=True)

# Identify categorical and numerical columns
categorical_cols = [0, 1, 3, 4, 5, 6, 8, 9, 11, 13]
numerical_cols = [2, 7, 10, 12]

# Fill missing values
from sklearn.impute import SimpleImputer
imputer_cat = SimpleImputer(strategy='most_frequent')
cc_apps[categorical_cols] = imputer_cat.fit_transform(cc_apps[categorical_cols])

imputer_num = SimpleImputer(strategy='mean')
cc_apps[numerical_cols] = imputer_num.fit_transform(cc_apps[numerical_cols])

# Encode categorical variables
le = LabelEncoder()
for col in categorical_cols:
    cc_apps[col] = le.fit_transform(cc_apps[col])

# Encode the target variable
cc_apps[13] = le.fit_transform(cc_apps[13])

# Split data into features and target
X = cc_apps.iloc[:, :-1]
y = cc_apps.iloc[:, -1]

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train models with GridSearchCV
results = {}

# Logistic Regression
param_grid_lr = {'C': [0.01, 0.1, 1, 10]}
grid_lr = GridSearchCV(LogisticRegression(max_iter=1000), param_grid_lr, cv=5)
grid_lr.fit(X_train_scaled, y_train)
results['LogisticRegression'] = grid_lr.best_score_

# Random Forest
param_grid_rf = {'n_estimators': [50, 100, 200], 'max_depth': [None, 5, 10]}
grid_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5)
grid_rf.fit(X_train_scaled, y_train)
results['RandomForest'] = grid_rf.best_score_

# SVM
param_grid_svm = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
grid_svm = GridSearchCV(SVC(), param_grid_svm, cv=5)
grid_svm.fit(X_train_scaled, y_train)
results['SVM'] = grid_svm.best_score_

# Select best model
best_model = grid_rf.best_estimator_

# Predict on test set
y_pred = best_model.predict(X_test_scaled)

# Get accuracy on test set
best_score = accuracy_score(y_test, y_pred)

# Print results
print("Model scores (cross-validation):", results)
print("Best model test accuracy:", best_score)
